In [1]:
# import libraries

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import faraway.datasets.ozone as ozone
import faraway.datasets.prostate as prostate 

In [104]:
data = ozone.load()  # El objeto data contiene los datos para el problema 1

In [105]:
## Guardar la información en:
# --> train: partición de entrenamiento    (incluyendo todas las variables)
# --> test: partición para la validacíon   (incluyendo todas las variables)

## Aquí va su código 

train, test = None, None    # Modifique estos objetos

# your code here
train, test = train_test_split(data, test_size=0.2, random_state=123)

In [106]:
# Debe entregar un vector booleano (True/False) que indique si la variable es significativa (True) o no (False).
# El orden de las variables en el vector debe ser: [const, vh, wind, humidity, temp, ibh, dpg, ibt, vis, doy]
# const = intercepto. 
# El vector debe tener por nombre sigVars
# Ejemplo: sigVars = [True, False, True, False, True, False, True, False, True, False]

## Aquí va su código 

sigVars = None # Modifique este objeto 

# your code here
X_train = train.drop(columns=["O3"])
X_train = sm.add_constant(X_train)
y_train = train["O3"]
X_test = test.drop(columns=["O3"])
X_test = sm.add_constant(X_test)
y_test = test["O3"]
model_lr_ols = sm.OLS(y_train, X_train).fit()
sigVars = model_lr_ols.pvalues < 0.05
sigVars

const       False
vh          False
wind        False
humidity     True
temp         True
ibh         False
dpg         False
ibt          True
vis         False
doy          True
dtype: bool

In [107]:
# Guarde el coeficiente de determinación ($R^2$) en un objeto númerico de nombre R2.
# En caso de redondear, considerar al menos 4 decimales de precisión

R2 = None # Modifique este objeto 

# your code here
R2 = model_lr_ols.rsquared
R2 = round(R2, 4)
R2

np.float64(0.6938)

In [108]:
# Guardar el error cuadrático medio en un objeto númerico de nombre MSE
MSE = None # Modifique este objeto 

# your code here
y_pred = model_lr_ols.predict(X_train)
MSE = metrics.mean_squared_error(y_train, y_pred)
MSE

19.55081161581369

In [109]:
# your code here
X2_train = train.drop(columns=["O3"])
y2_train = train["O3"]
X2_test = test.drop(columns=["O3"])
y2_test = test["O3"]
X2_train = X2_train[['humidity', 'temp', 'ibt', 'doy']]
X2_test = X2_test[['humidity', 'temp', 'ibt', 'doy']]

model_lr_ols2 = sm.OLS(y2_train, X2_train).fit()

AIC = []
AIC.append(model_lr_ols.aic)
AIC.append(model_lr_ols2.aic)

if AIC[0] < AIC[1]:
    print("Modelo 1 has a lower AIC")
    preferible = False
else:
    print("Modelo 2 has a lower AIC")
    preferible = True

print(AIC)
print(preferible)

Modelo 1 has a lower AIC
[np.float64(1554.0759808495568), np.float64(1597.0207391599592)]
False


In [110]:
AIC_forward = None                # Variable que debe modificar
variables_seleccionadas = None    # Variable que debe modificar
preferible_forward = None         # Variable que debe modificar

all_columns = ['vh','wind','humidity','temp','ibh','dpg','ibt','vis','doy']
selected_model2_columns = ['temp', 'ibt', 'doy']
rest_columns = list(set(all_columns) - set(selected_model2_columns))

X3_train = train[selected_model2_columns].copy()
y3_train = train["O3"].copy()
X_full = train[all_columns].copy()

model_lr_ols3 = sm.OLS(y3_train, X3_train).fit()
aic_curr = model_lr_ols3.aic
selected = selected_model2_columns.copy()

improved = True
while improved and len(rest_columns) > 0:
    improved = False
    best_var = None
    best_aic = aic_curr
    best_model = model_lr_ols3

    for var in rest_columns:
        cols_try = selected + [var]
        model_try = sm.OLS(y3_train, X_full[cols_try]).fit()
        if model_try.aic < best_aic:
            best_aic = model_try.aic
            best_var = var
            best_model = model_try

    if best_var is not None:
        selected.append(best_var)
        rest_columns.remove(best_var)
        model_lr_ols3 = best_model
        aic_curr = best_aic
        improved = True

AIC_forward = float(aic_curr)

variables_seleccionadas = selected
testF = [model_lr_ols3.fvalue, model_lr_ols3.f_pvalue]
preferible_forward = True
print("preferible_forward:", preferible_forward)
print("AIC_forward:", AIC_forward)
print("variables_seleccionadas:", variables_seleccionadas)
print(f"Test F -> F={testF[0]:.4f}, p-value={testF[1]:.4g}")

preferible_forward: True
AIC_forward: 1547.6899975652686
variables_seleccionadas: ['temp', 'ibt', 'doy', 'vh', 'humidity', 'vis']
Test F -> F=410.7837, p-value=6.735e-129


In [111]:
pred = None         # Variable que debe modificar
MSEpred = None      # Variable que debe modificar

# your code here
X3_test = test[variables_seleccionadas].copy()
y3_test = test["O3"].copy()
pred = model_lr_ols3.predict(X3_test)
MSEpred = float(np.mean((y3_test - pred) ** 2))
MSEpred

17.658748987607325

In [112]:
prostate_data = prostate.load()  # cargamos los datos del problema 2

In [113]:
## Guardar la información en:
# --> train_prostate: partición de entrenamiento    (incluyendo todas las variables)
# --> test_prostate: partición para la validacíon   (incluyendo todas las variables)

## Aquí va su código 

train_prostate, test_prostate = None, None    # Modifique estos objetos

# your code here
train_prostate, test_prostate = train_test_split(prostate_data, test_size=0.2, random_state=123)

In [114]:
# Aquí va su código

porcentaje_clasificacion = None # Variable que debe modificar

# your code here
X_train = train_prostate.drop(columns="svi", axis=1)
y_train = train_prostate["svi"]
X_test = test_prostate.drop(columns="svi", axis=1)
y_test = test_prostate["svi"]

model_logit = sm.Logit(y_train, X_train).fit()

y_pred_prob = model_logit.predict(X_test)
y_pred_class = (y_pred_prob >= 0.5).astype(int)
accuracy = metrics.accuracy_score(y_test, y_pred_class)
porcentaje_clasificacion = accuracy
porcentaje_clasificacion

Optimization terminated successfully.
         Current function value: 0.162442
         Iterations 9


0.7

In [115]:
# Aquí va su código

VIF_vars = None # Variable que debe modificar

# your code here
vars_order = ['lcavol', 'lweight', 'age', 'lbph', 'lcp', 'gleason', 'pgg45', 'lpsa']

X_vif = X_train[vars_order].copy()
VIF_vars = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

VIF_vars = [float(v) for v in VIF_vars] 
VIF_vars

[6.749196983258511,
 57.783939331393654,
 82.40542217672011,
 1.195335723452414,
 2.531763153388447,
 84.90034919124903,
 3.809147884382757,
 16.916887569167937]

In [116]:
# Aquí va su código

porcentaje_clasificacion_backward = None # Variable que debe modificar

# your code here
df_vif = pd.DataFrame({
    "Features": vars_order,
    "VIF": VIF_vars
})

df_vif_filtered = df_vif[df_vif["VIF"] < 10]
selected_feats = df_vif_filtered['Features'].tolist()

while True:
    Xb_train = sm.add_constant(train_prostate[selected_feats], has_constant="add")
    logit_fit = sm.Logit(y_train, Xb_train).fit(disp=0)

    pvals = logit_fit.pvalues.drop(labels=["const"], errors="ignore")

    if pvals.empty or (pvals.max() < 0.05):
        break

    worst_feat = pvals.idxmax()
    selected_feats.remove(worst_feat)

Xb_test = sm.add_constant(test_prostate[selected_feats], has_constant="add")
y_pred_prob_b = logit_fit.predict(Xb_test)
y_pred_class_b = (y_pred_prob_b >= 0.5).astype(int)
accuracy_b = metrics.accuracy_score(y_test, y_pred_class_b)
porcentaje_clasificacion_backward = accuracy_b
porcentaje_clasificacion_backward

0.75